# Generate SMA using dedicated service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *simple moving average - SMA* from a dedicated data service.

### Inputs/outputs
Simple moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

Basically, SMA is formed by computing the average price of a security over a specific number of points. As its name implies, simple moving average is an average that moves. Old data is dropped and new data becomes available, causing the average to move along the time.

N-points SMA data point is computed by adding up the last N period's trade prices and then dividing by N as follows:

$$ SMA = \frac{\Sigma Price n}{N} $$
Where:
* *Price n* is the trade price of the *period n*
* *N* is the count of time periods

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve SMA indicator from the hosted service. The queried endpoint in this script are:
* *TickSmaService*: to directly retrieve SMA data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run SMA sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_sma_pb2 as tick_sma
import systemathics.apis.services.tick_analytics.v1.tick_sma_pb2_grpc as tick_sma_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request
To request *Tick SMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Simple moving average request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 SMA parameters

In [ ]:
field = tick_sma.SMA_PRICE_TRADE; # set the SMA calculation field
long_sma = 200; # set the long SMA length
short_sma = 50; # set the short SMA length
sampling = 1 * 60

#### 3.3 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 2, day = 18), 
    end_date = date.Date(year = 2020, month = 2, day = 18)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 8, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.4 Long SMA request creation
The following code snippets creates *gRPC client*, process *long SMA* request and returns the request reply:

In [ ]:
# generate the tick long SMA request
long_request = tick_sma.TickSmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    field = field,
    length = long_sma,
    sampling = duration.Duration(seconds = sampling)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the tick SMA service
    service = tick_sma_service.TickSmaServiceStub(channel)
    
    long_averages = []
    # process the tick long SMA request
    for l in service.TickSma(request = long_request, metadata = [('authorization', token)]):
        if l.average.value > 0:  # we have an average
            long_averages.append(l.average.value)
        else: 
            long_averages.append(None)

#### 3.4 Short SMA request creation
The following code snippets creates *gRPC client*, process *short SMA* request and returns the request reply:

In [ ]:
# generate the tick short SMA request
short_request = tick_sma.TickSmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    field = field,
    length = short_sma,
    sampling = duration.Duration(seconds = sampling)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the tick SMA service
    service = tick_sma_service.TickSmaServiceStub(channel)
    
    time_stamps, short_averages, prices = [], [], []
    # process the tick short SMA request
    for s in service.TickSma(request = short_request, metadata = [('authorization', token)]):
        time_stamps.append(s.time_stamp)
        prices.append(s.value)
        if s.average.value > 0:  #we have an average
            short_averages.append(s.average.value)
        else: 
            short_averages.append(None)

### Step 4: Visualize data

#### 4.1 Retrieve SMAs data

In [ ]:
# create the SMAs dataframe
dates=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
d = {'Date': dates, 'Long Sma': long_averages, 'Short Sma': short_averages, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.4 Plot SMAs and prices

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Short Sma'], name='Short Sma', marker_color ='green'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Long Sma'], name='Long Sma', marker_color ='red'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick SMAs and prices for {0}-{1}'.format(ticker,exchange))
fig.show()